#step1 : 데이터 업로드

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip /content/drive/MyDrive/dataset.zip

In [ ]:
# cnn 환경 세팅
%cd /content/drive/MyDrive/project/CNN

In [ ]:
# 데이터 확인하기
from glob import glob
import torch
from IPython.display import Image, clear_output

img_list = glob('/content/dataset/*.jpg')
print(len(img_list))

txt_list = glob('/content/dataset/*.txt')
print(len(txt_list))

In [ ]:
!pwd

In [ ]:
import os

os.mkdir('/content/drive/MyDrive/project/CNN/project_custom')
os.mkdir('/content/drive/MyDrive/project/CNN/project_custom/images')
os.mkdir('/content/drive/MyDrive/project/CNN/project_custom/labels')

In [ ]:
# 이미지 파일 이동
import shutil

for img in img_list:
  shutil.copy(img, './project_custom/images')

In [ ]:
# 텍스트 파일 이동
import shutil

for txt in txt_list:
  shutil.copy(txt, './project_custom/labels')

In [ ]:
# 데이터셋 디렉토리 경로 설정
dataset_dir = "/content/drive/MyDrive/project/CNN/project_custom"
dataset_images_dir = "/content/drive/MyDrive/project/CNN/project_custom/images"
dataset_lables_dir = "/content/drive/MyDrive/project/CNN/project_custom/labels"

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from PIL import Image
from glob import glob

In [ ]:
# 이미지 데이터 정규화를 위한 변환
transform = transforms.Compose([
    transforms.ToTensor(),  # 이미지를 PyTorch Tensor로 변환
    transforms.Normalize((0.5,), (0.5,))  # 이미지를 0과 1 사이의 값으로 정규화
])

In [ ]:
# 사용자 정의 데이터셋 클래스
class CustomDataset(Dataset):
    def __init__(self, image_names, labels, transform=None):
        self.image_names = image_names
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, index):
        image_name = self.image_names[index]
        image_path = os.path.join(dataset_dir, f"{image_name}.jpg")
        image = Image.open(image_path)

        # 이미지 데이터 정규화
        if self.transform:
            image = self.transform(image)

        # 라벨 데이터 전처리 (필요한 경우 추가)
        label = self.labels[index]

        return image, label


In [ ]:
# 데이터셋 로딩 함수
def load_dataset():
    # 이미지 파일들과 텍스트 파일들의 이름을 리스트로 가져오기
    # 이미지 파일들의 이름 리스트
    jpg_list = glob(os.path.join(dataset_images_dir, "*.jpg"))
    image_names = [os.path.splitext(os.path.basename(path))[0] for path in jpg_list]

    # 텍스트 파일들의 이름 리스트
    txt_list = glob(os.path.join(dataset_lables_dir, "*.txt"))
    txt_names = [os.path.splitext(os.path.basename(path))[0] for path in txt_list]

    return image_names, txt_names

In [ ]:
# 데이터셋 로딩
image_names, labels = load_dataset()

# 데이터셋 생성
custom_dataset = CustomDataset(image_names, labels, transform)

In [ ]:
# 데이터 로더 생성
batch_size = 32
data_loader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
import torch
import torch.nn as nn

class SimpleCNN(nn.Module):
    def __init__(self, 클래스_개수):
        super(SimpleCNN, self).__init__()
        self.합성곱_계층들 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.완전연결_계층들 = nn.Sequential(
            nn.Linear(64 * 28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 클래스_개수)
        )

    def forward(self, x):
        x = self.합성곱_계층들(x)
        x = x.view(x.size(0), -1)
        x = self.완전연결_계층들(x)
        return x

In [ ]:
num_classes = 10  # 분류하려는 클래스 개수에 맞게 설정
model = SimpleCNN(num_classes)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9) # 옵티마이저 선택 (SGD 사용)

In [ ]:
# 손실 함수 정의 (CrossEntropyLoss 사용)
criterion = nn.CrossEntropyLoss()

# 학습 루프 설정 및 학습 실행
num_epochs = 10  # 원하는 에폭 수에 맞게 설정

In [ ]:
for epoch in range(num_epochs):
    for images, labels in data_loader:
        images, labels = images.to(device), labels.to(device)

        # 순전파
        outputs = model(images)
        loss = criterion(outputs, labels)

        # 역전파 및 옵티마이저 업데이트
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")